<a href="https://colab.research.google.com/github/pedrov718/Content-Based-Book-Rec-System/blob/main/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import pandas as pd
import spacy 
import re
import string
import nltk
import numpy as np
import pprint
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
df  = pd.read_csv('/content/books_with_blurbs.csv')

In [4]:
df.head()

,ISBN,Title,Author,Year,Publisher,Blurb
0,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"Here, for the first time in paperback, is an o..."
1,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"The fascinating, true story of the world's dea..."
2,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,Winnie and Helen have kept each others worst s...
3,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,Historians and inquisitive laymen alike love t...
4,1881320189,Goodbye to the Buttermilk Sky,Julia Oliver,1994,River City Pub,This highly praised first novel by fiction wri...


In [5]:
df['Blurb'][10:15]

10    Newly graduated from college, Sam Altman is gr...
11    Three passengers are dead. Fifty-six are injur...
12    In an Arizona desert, a man wanders in a daze,...
13    Dr. Ransom is abducted by a megalomaniacal phy...
14    A novel of startling scope and ambition, , dep...
Name: Blurb, dtype: object

In [129]:
# Load spacy
nlp = spacy.load('en_core_web_sm')

def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    #Remove stop words
    text = text.split()
    # useless_words = nltk.corpus.stopwords.words("english")
    # useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text]

    #Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string


In [130]:
print("Cleaning train data...\n")

df["blurb_clean"] = df['Blurb'].apply(lambda x: clean_string(x, stem='Lem'))

Cleaning train data...



In [131]:
df.head()

,ISBN,Title,Author,Year,Publisher,Blurb,blurb_clean,Title_lower,blurb_clean_title
0,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"Here, for the first time in paperback, is an o...",Here for the first time in paperback is an out...,decision in normandy,decision in normandy here for the first time i...
1,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"The fascinating, true story of the world's dea...",The fascinating true story of the world deadli...,flu: the story of the great influenza pandemic...,flu: the story of the great influenza pandemic...
2,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,Winnie and Helen have kept each others worst s...,Winnie and Helen have kept each others worst s...,the kitchen god's wife,the kitchen god's wife winnie and helen have k...
3,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,Historians and inquisitive laymen alike love t...,Historians and inquisitive layman alike love t...,what if?: the world's foremost military histor...,what if?: the world's foremost military histor...
4,1881320189,Goodbye to the Buttermilk Sky,Julia Oliver,1994,River City Pub,This highly praised first novel by fiction wri...,This highly praised first novel by fiction wri...,goodbye to the buttermilk sky,goodbye to the buttermilk sky this highly prai...


In [132]:
df['Title_lower'] = df['Title'].apply(lambda x: x.lower())

In [133]:
df['blurb_clean_title'] =  df['Title_lower'] +  ' ' + df['blurb_clean']

In [134]:
df['blurb_clean_title'][10]

'wild animus Newly graduated from college Sam Altman is gripped by an inexplicable urge to lose himself in the wilderness and team up with an enigmatic young woman who seems bent on helping him realize his dream'

In [135]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [136]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['blurb_clean_title'])

In [137]:
sentence = 'it was the best of times it was the worst time'
clean_string(sentence)

'it was the best of times it was the worst time'

In [138]:
def calculate_cosine_sim():
  sentence = [input("Write a line from your favorite book? ")]
  y = vectorizer.transform(sentence)
  cosine_sim = cosine_similarity(X,y)
  max_index = np.argmax(cosine_sim)
  return print('\n', df['Title'].iloc[max_index], '\n', df['Blurb'].iloc[max_index])

In [139]:
calculate_cosine_sim()

Write a line from your favorite book? pizza is my favorite color

 The Pizza Book: Everything There Is To Know About the World's Greatest Pie 
 This young chef makes pizza making easy and fun. More than 200 easy-to-follow recipes. Supplementing the recipes is advice about ingredients, equipment and technique.
